In [1]:
import numpy as np
import quaternion

from intrepid_environment.quadcopter_environment import QuadcopterController

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
# initialize RPC Drone Controller
DT_MS = 100

quadcopter_rpc_controller = QuadcopterController(DT_MS)

Setup trajectory for drone to follow.

In [4]:
from quadcopter_control.trajectory import Trajectory, TrajectoryPoint

In [5]:
async def get_trajectory_with_initial_position():
    sim_time = await quadcopter_rpc_controller.simulation_time()
    initial_orientation = quadcopter_rpc_controller.rotation_angles()
    initial_attitude = quaternion.from_euler_angles(*initial_orientation)
    initial_omega = quadcopter_rpc_controller.angular_velocity()
    initial_position = quadcopter_rpc_controller.position()
    initial_linear_velocity = quadcopter_rpc_controller.linear_velocity()
    initial_acceleration = quadcopter_rpc_controller.acceleration()

    starting_point = TrajectoryPoint(
        time=sim_time,
        position=initial_position,
        velocity=initial_linear_velocity,
        accel=initial_acceleration,
        attitude=initial_attitude,
        omega=initial_omega,
    )

    trajectory = Trajectory()
    trajectory.add_point(starting_point)

    return trajectory

In [6]:
def get_next_trajectory_point(trajectory, sim_time, dt, dp):
    pt = trajectory.next_trajectory_point(sim_time + dt)
    pt.position += dp
    return pt

In [7]:
trajectory_time_offset = 0
dp = np.zeros(3)

In [8]:
from quadcopter_control.controller import QuadcopterController, calculate_drone_moment_of_inertia

In [9]:
DRONE_HEIGHT = 0.028
DRONE_RADIUS = 0.05
DRONE_MASS = 0.041

In [10]:
(Ixx, Iyy, Izz) = calculate_drone_moment_of_inertia(DRONE_HEIGHT, DRONE_RADIUS, DRONE_MASS)
(Ixx, Iyy, Izz)

(2.830366666666667e-05, 2.830366666666667e-05, 5.125000000000001e-05)

## Scenario 0: Zero Gains

Test whether controller runs without controller gains set.

In [11]:
trajectory = await get_trajectory_with_initial_position()

In [12]:
quadcopter_controller = QuadcopterController(
    DT_MS / 1000.0,
    DRONE_MASS,
    Ixx, Iyy, Izz,
    max_tilt_angle=0.52,
    max_ascent_rate=10.0
)

In [ ]:
trajectory = await get_trajectory_with_initial_position()

async def control_step():
    sim_time = await quadcopter_rpc_controller.simulation_time()
    quadcopter_controller.run_control(
        traj_pt=trajectory.next_trajectory_point(sim_time+DT_MS),
        est_pos=quadcopter_rpc_controller.position(),
        est_vel=quadcopter_rpc_controller.linear_velocity(),
        est_omega=quadcopter_rpc_controller.angular_velocity(),
        est_att=quaternion.from_euler_angles(
            *quadcopter_rpc_controller.rotation_angles()
        ),
    )

In [14]:
await control_step()

## Scenario 1: Altitude Control

Tune m parameter until drone stays in place. This scenario using very basic Motor Thrust Controller that equaly distributes thrust to 4 motors.

In [ ]:
# trajectory = Trajectory()

# trajectory.add_point(TrajectoryPoint(
#     time=sim_time,
#     position=np.array([0.0, 0.0, 0.0]),
#     velocity=np.zeros(3),
#     accel=np.zeros(3),
#     attitude=np.quaternion(1.0, 0.0, 0.0, 0.0),
#     omega=np.array([0.0, 0.0, 0.0])
# ))

# # add 500 datapoints so drone stays in place

# for i in range(0, 5000):
#     trajectory.add_point(TrajectoryPoint(
#         time=(sim_time + i*DT_MS),
#         position=np.array([0.0, 5.0, 0.0]),
#         velocity=np.zeros(3),
#         accel=np.zeros(3),
#         attitude=np.quaternion(1.0, 0.0, 0.0, 0.0),
#         omega=np.array([0.0, 0.0, 0.0])
#     ))

In [ ]:
# # position controller is turned off
# kp_pos_xy = 0
# kp_pos_z = 0
# kp_vel_xy = 0
# kp_vel_z = 0

# kappa = 2.0

# # tune these
# ki_pos_z = 50
# kp_yaw = 2
# kp_bank = 10
# kp_pqr = np.array([95.0, 95.0, 6.0])

# # trajectory dt
# dp = np.zeros(3)

In [ ]:
# await run_simulation(n_episodes=200)